In [ ]:

import cv2
import numpy as np
def stitch(image):
    # image stitching
    stitcher = cv2.Stitcher_create()
    status, stitched = stitcher.stitch(image)
 
    # black border processing
    if status == cv2.Stitcher_OK:
        # contour extraction
        stitched = cv2.copyMakeBorder(stitched, 50, 50, 50, 50, cv2.BORDER_CONSTANT, (0, 0, 0))
        gray = cv2.cvtColor(stitched, cv2.COLOR_BGR2GRAY)
        thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY)[1]
        cnts = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)[0]
        cv2.imwrite('stitched_1.jpg', thresh)

        # the smallest rectangle
        mask = np.zeros(thresh.shape, dtype="uint8")
        (x, y, w, h) = cv2.boundingRect(cnts[0])
        cv2.rectangle(mask, (x, y), (x + w, y + h), 255, -1)
        cv2.imwrite('stitched_2.jpg', mask)

        # erosion treatment until minRect pixel value is 0
        minRect = mask.copy()
        sub = mask.copy()
        while cv2.countNonZero(sub) > 0:
            minRect = cv2.erode(minRect, None)
            sub = cv2.subtract(minRect, thresh)
        cv2.imwrite('stitched_3.jpg', minRect)

        # Extract the minRect contour and crop it
        cnts = cv2.findContours(minRect, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)[0]
        (x, y, w, h) = cv2.boundingRect(cnts[0])
        stitched = stitched[y:y + h, x:x + w]
 
        cv2.imwrite('stitched_4.jpg', stitched)
        cv2.waitKey(0)
        cv2.destroyAllWindows()
    else:
        print('Image matching feature points are insufficient')
 
 
if __name__ == "__main__":
    image1 = cv2.imread('/a.jpg')
    image2 = cv2.imread('/b.jpg')
    image = image1, image2
    stitch(image)